In [ ]:
import numpy as np
from bqplot import pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

In [ ]:
Tmax = 20
n_timesteps = 200
delta_t = Tmax / n_timesteps

noise = np.random.normal(0, 1, (3, n_timesteps))

Y = np.zeros( (3, n_timesteps))
#setting the start values (arbitrary)
Y[0, 0] = 11.1
Y[1, 0] = 12.
Y[2, 0] = 11.

sigma = np.array([(0.2,0.,0.),(0.0375,0.1452,0),(0.025,0.0039,0.0967)])
Omega = sigma.dot(sigma.transpose())

a = np.array([0., -1, 0, 1])
A = np.diag(a[1:])
#a[0] = -np.sum([ a[j]*np.log(Y[j-1, 0]) for j in range(1,4)])
delta = np.array([1,1,0])

alpha = np.zeros(n_timesteps-1)
wealth = np.zeros(n_timesteps)
pi_ast = np.zeros((3, n_timesteps-1))

def calc_Y (a, delta):
    for i in range(n_timesteps-1):
        alpha_t = a[0] + sum([ a[j]*np.log(Y[j-1, i]) for j in range(1,4)])
        #print(alpha_t)
        alpha[i] = alpha_t
        for j in range(3):
            dY = delta[j]*Y[j, i]*alpha_t + Y[j, i]*sum( sigma[j,j2]*noise[j2, i] for j2 in range(3))
            Y[j, i+1] = Y[j, i] + delta_t* dY
        #alpha_val = a[0] + np.dot( a[1:], np.log(Y[:, i]))
        tau = Tmax - i*delta_t
        pi_ast[:, i] = np.dot( np.linalg.inv(Omega), delta)*alpha_t +  np.dot( delta.transpose(), np.dot(Omega, delta)) *(tau*alpha_t/2 + tau**2/4*np.trace(np.dot(A, Omega)))*a[1:]
        dwealth = sum( [pi_ast[j, i]*(Y[j, i+1] - Y[j, i]) for j in range(3)] )
        wealth[i+1] = wealth[i] + dwealth #np.sum(pi_ast)
    return Y

#plt.plot(alpha)
#alpha

#Y = calc_Y(a, delta)

#plt.figure()
#plt.plot(Y[0, :], label = "stock 1")
#plt.plot(Y[1, :], label = "stock 2", colors=['orangered'])
#plt.plot(Y[2, :], label = "stock 2", colors=['green'])
#plt.show()

reset_zero = widgets.Button(description='Reset to 0')
reset_exam = widgets.Button(description='Reset to ex')
new_chance = widgets.Button(description='New chance')

slider_a0 = widgets.FloatSlider(min=-1.5, max=1.5, step=0.05, continuous_update= False, description='a0:', value= 0)
slider_a1 = widgets.FloatSlider(min=-1.5, max=1.5, step=0.05, continuous_update= False, description='a1:', value=-1)
slider_a2 = widgets.FloatSlider(min=-1.5, max=1.5, step=0.05, continuous_update= False, description='a2:', value= 0)
slider_a3 = widgets.FloatSlider(min=-1.5, max=1.5, step=0.05, continuous_update= False, description='a3:', value= 1)

slider_delta1 = widgets.FloatSlider(min=-1.5, max=1.5, step=0.05, continuous_update= False, description='d1:', value=1)
slider_delta2 = widgets.FloatSlider(min=-1.5, max=1.5, step=0.05, continuous_update= False, description='d2:', value=1)
slider_delta3 = widgets.FloatSlider(min=-1.5, max=1.5, step=0.05, continuous_update= False, description='d3:', value=0)

controls_a = widgets.HBox( [slider_a1, slider_a2, slider_a3])
controls_d = widgets.HBox( [slider_delta1, slider_delta2, slider_delta3])

auto_a0_box = widgets.Checkbox(value=False, description='auto a0')

controls = widgets.VBox ( [widgets.HBox([slider_a0, auto_a0_box, reset_zero, reset_exam, new_chance]), controls_a, controls_d])

plot_output = widgets.Output()
plot_output.clear_output()

def slider_changed(change):
    if auto_a0_box.value:
        a0_aut = -np.sum([ [slider_a1.value, slider_a2.value, slider_a3.value][j]*np.log([11.1, 12., 11.][j]) for j in range(3)])
        slider_a0.disabled = True
        slider_a0.value = a0_aut
        Y = calc_Y(np.array([a0_aut, slider_a1.value, slider_a2.value, slider_a3.value]), 
               np.array([slider_delta1.value, slider_delta2.value, slider_delta3.value]) )
    else:
        slider_a0.disabled = False
        Y = calc_Y(np.array([slider_a0.value, slider_a1.value, slider_a2.value, slider_a3.value]), 
               np.array([slider_delta1.value, slider_delta2.value, slider_delta3.value]) )
    plot_output.clear_output(wait=True)
    with plot_output:
        #fig, ((ax1, ax2)) = plt.subplots(1,2, figsize=(20,5))
        plt.figure()
        plt.title('Stock prices')
        plt.plot(Y[0, :], label = "stock 1")
        plt.plot(Y[1, :], label = "stock 2", colors=['orangered'])
        plt.plot(Y[2, :], label = "stock 2", colors=['green'])
        plt.show()
        #ax1.plot(Y[1, :], label = "stock 2")
        #ax1.plot(Y[2, :], label = "stock 3")
        plt.figure()
        plt.title('alpha')
        plt.plot(alpha)
        plt.show()
        
        plt.figure()
        plt.title('wealth')
        plt.plot(wealth)
        plt.show()

for s in [slider_delta1, slider_delta2, slider_delta3, slider_a0, slider_a1, slider_a2, slider_a3, auto_a0_box]:
    s.observe(slider_changed, names='value')

def reset_zero_fun (change):
    auto_a0_box.value = False
    slider_a0.value = 0
    slider_a1.value = 0
    slider_a2.value = 0
    slider_a3.value = 0
    slider_delta1.value = 0
    slider_delta2.value = 0
    slider_delta3.value = 0
reset_zero.on_click(reset_zero_fun)

def reset_exam_fun (change):
    auto_a0_box.value = False
    slider_a0.value = 0
    slider_a1.value = -1
    slider_a2.value = 0
    slider_a3.value = 1
    slider_delta1.value = 1
    slider_delta2.value = 1
    slider_delta3.value = 0
reset_exam.on_click(reset_exam_fun)

def new_chance_fun (change):
    newnoise = np.random.normal(0, 1, (3, n_timesteps))
    for i in range(3):
        for j in range(n_timesteps):
            noise[i,j] = newnoise[i,j]
    slider_changed({})
new_chance.on_click(new_chance_fun)

slider_changed({})

display(controls, plot_output)

In [ ]:
n_samples = 2500
wealth_results = np.zeros(n_samples)

plot_output = widgets.Output()
plot_output.clear_output()
display(plot_output)

for samplei in range(n_samples):
    newnoise = np.random.normal(0, 1, (3, n_timesteps))
    for i in range(3):
        for j in range(n_timesteps):
            noise[i,j] = newnoise[i,j]
    calc_Y(np.array([slider_a0.value, slider_a1.value, slider_a2.value, slider_a3.value]), 
               np.array([slider_delta1.value, slider_delta2.value, slider_delta3.value]) )
    #print(wealth[-1])
    wealth_results[samplei] = wealth[-1]
    if samplei % 250 == 0 and samplei > 0:
        plot_output.clear_output(wait = True)
        with plot_output:
            plt.figure(padding_y=0)
            hist = plt.bin(wealth_results[:samplei], padding=0, density= True)
            hist.bins = 100
            #hist.min=-5
            #hist.max=80
            plt.show()
            print("Mean : ", np.mean( wealth_results[:samplei]))
            print("standard deriv", np.std(wealth_results[:samplei]))
            print("Sharpe ratio: ", np.mean( wealth_results[:samplei])/ np.std(wealth_results[:samplei]))
            
    
#print(wealth_results)